<font size=6><b>펀드 운용모델

In [1]:
from IPython.display import Image
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
warnings.filterwarnings(action='ignore')
sns.set()

#-------------------- 차트 관련 속성 (한글처리, 그리드) -----------
#plt.rc('font', family='NanumGothicOTF') # For MacOS
plt.rcParams['font.family']= 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False


#-------------------- 주피터 , 출력결과 넓이 늘리기 ---------------
from IPython.core.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('max_colwidth', None)



# Data Load

## K5 excel

In [2]:
sample_df = pd.read_excel("./실습07_펀드_운용모델.xlsx", sheet_name='펀드', index_col=0)
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 253 entries, nan to 252.0
Data columns (total 21 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   편입비중         253 non-null    object 
 1   Unnamed: 2   253 non-null    object 
 2   Unnamed: 3   253 non-null    object 
 3   Unnamed: 4   253 non-null    object 
 4   Unnamed: 5   253 non-null    object 
 5   지수           252 non-null    float64
 6   (Random)     253 non-null    object 
 7   Unnamed: 8   253 non-null    object 
 8   총좌수          252 non-null    float64
 9   iNAV         252 non-null    float64
 10  AUM          252 non-null    float64
 11  일자별 매매내역     253 non-null    object 
 12  Unnamed: 13  253 non-null    object 
 13  Unnamed: 14  253 non-null    object 
 14  Unnamed: 15  253 non-null    object 
 15  Unnamed: 16  253 non-null    object 
 16  주식 잔고        253 non-null    object 
 17  Unnamed: 18  253 non-null    object 
 18  Unnamed: 19  253 non-null    object 
 19  Un

In [3]:
sample_df.head()

,편입비중,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,지수,(Random),Unnamed: 8,총좌수,iNAV,AUM,일자별 매매내역,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,주식 잔고,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21
일자,,,,,,,,,,,,,,,,,,,,,
NaN,A전자,B건설,C증권,D화학,E엔터,NaN,설정좌수,환매좌수,NaN,NaN,NaN,A전자,B건설,C증권,D화학,E엔터,A전자,B건설,C증권,D화학,E엔터
1.0,0.060241,0.072289,0.395181,0.070683,0.401606,100.000000,1000000,0,1000000.0,10000.000000,1.000000e+10,60240.963855,36144.578313,131726.907631,7068.273092,4016.064257,60240.963855,36144.578313,131726.907631,7068.273092,4016.064257
2.0,0.06055,0.071976,0.401645,0.069917,0.395912,98.872776,100000,0,1100000.0,9887.277590,1.087601e+10,6024.096386,3614.457831,13172.690763,706.827309,401.606426,66265.060241,39759.036145,144899.598394,7775.100402,4417.670683
3.0,0.062687,0.072527,0.40558,0.069572,0.389634,97.549515,100000,0,1200000.0,9754.951486,1.170594e+10,6024.096386,3614.457831,13172.690763,706.827309,401.606426,72289.156627,43373.493976,158072.289157,8481.927711,4819.277108
4.0,0.062501,0.072085,0.397389,0.067955,0.400069,97.279693,100000,150000,1150000.0,9727.969317,1.118716e+10,-3012.048193,-1807.228916,-6586.345382,-353.413655,-200.803213,69277.108434,41566.26506,151485.943775,8128.514056,4618.473896


# 시총 TOP-N 업종명, 업종코드, 종가
* 20200101 ~ 20221231

In [4]:
from pykrx import stock 

__START_DATE = "20200101"
__END_DATE   = "20221231"
__TOPN = 7

mc_df = stock.get_market_cap("20221229")[:__TOPN]
print(mc_df)
top_idx_list   = mc_df.index
print(top_idx_list)
for ticker in top_idx_list:
    티커명 = stock.get_market_ticker_name(ticker)
    print(ticker, 티커명)

            종가             시가총액       거래량          거래대금       상장주식수
티커                                                                 
005930   55300  330128975015000  11295935  628653753150  5969782550
373220  435500  101907000000000    265226  116156988000   234000000
207940  821000   58433854000000     35179   28994143000    71174000
000660   75000   54600177375000   1736661  130328025000   728002365
051910  600000   42355405800000    103968   62525555000    70592343
005935   50500   41555778350000    722569   36552249900   822886700
006400  591000   40639837230000    197767  118176938000    68764530
Index(['005930', '373220', '207940', '000660', '051910', '005935', '006400'], dtype='object', name='티커')
005930 삼성전자
373220 LG에너지솔루션
207940 삼성바이오로직스
000660 SK하이닉스
051910 LG화학
005935 삼성전자우
006400 삼성SDI


In [5]:
ticker_name_list = []
for ticker in top_idx_list:
    티커명 = stock.get_market_ticker_name(ticker)
    ticker_name_list.append(티커명)
ticker_name_list 

['삼성전자', 'LG에너지솔루션', '삼성바이오로직스', 'SK하이닉스', 'LG화학', '삼성전자우', '삼성SDI']

In [6]:
top_idx_list     = ['005930', '207940', '000660', '051910', '006400']
ticker_name_list = ['삼성전자', '삼성바이오로직스', 'SK하이닉스', 'LG화학', '삼성SDI']

# 종목 기본정보 
* 시가총액 = 주가 * 발행주식수(유동주식수 + 자기주식수)
* 유통주식수  = 발행주식총수 - 자기주식수(대주주+우리사주+자사주+보호예수+정부기관(5%이상)+해외DR)
* 유통주식비율 = 유통주식수/발행주식총수 * 100

In [7]:
import requests
from bs4 import BeautifulSoup

def my_craw(ticker="005930"):
    res = requests.get(f"https://comp.fnguide.com/SVO2/ASP/SVD_main.asp?pGB=1&gicode=A{ticker}&cID=&MenuYn=Y&ReportGB=&NewMenuID=11&stkGb=&strResearchYN=")
    soup     = BeautifulSoup(res.text, features="html.parser")
    
    #--- 보통주:발행주식수  ,  우선주
    # #svdMainGrid1 > table > tbody > tr:nth-child(7) > td:nth-child(2)
    find_str = soup.select_one("div#svdMainGrid1 > table > tbody > tr:nth-child(6) >  tr > td.r").text
    #print(find_str)
    split_str = find_str.split("/")
    발행주식수 = int(split_str[0].strip().replace(",",""))
    우선주 = int(split_str[1].strip().replace(",",""))
    
    #--- 유동주식수,비율
    find_str = soup.select_one("div#svdMainGrid1 > table > tbody > tr:nth-child(6) >  tr > td.cle.r").text
    #print(find_str)
    split_str = find_str.split("/")
    유동주식수 = int(split_str[0].strip().replace(",",""))
    유동비율      = float(split_str[1].strip())
    return 발행주식수, 유동주식수, 유동비율

temp_list = []
for i, ticker in enumerate(top_idx_list):
    발행주식수,유동주식수,유동비율 = my_craw(ticker)    
    ticker_name = ticker_name_list[i]
    temp_list.append((ticker, ticker_name, 발행주식수, 유동주식수, 유동비율))
종목기본정보 = pd.DataFrame(temp_list, columns=['ticker', 'ticker_name', '발행주식수','유동주식수','유동비율'])    
종목기본정보 = 종목기본정보.set_index("ticker_name")
종목기본정보

,ticker,발행주식수,유동주식수,유동비율
ticker_name,,,,
삼성전자,005930,5969782550,4525870769,75.81
삼성바이오로직스,207940,71174000,17623664,24.76
SK하이닉스,000660,728002365,537759511,73.87
LG화학,051910,70592343,47036786,66.63
삼성SDI,006400,68764530,50306427,73.16


In [8]:
#지수표현식 없애기 
pd.options.display.float_format = '{:.0f}'.format
종목기본정보['calc_유동주식비율'] = 종목기본정보['유동주식수']  / 종목기본정보['발행주식수'] * 100
종목기본정보['calc_유동주식수'] = 종목기본정보['발행주식수'] *  (종목기본정보['유동비율'] /100)
종목기본정보

,ticker,발행주식수,유동주식수,유동비율,calc_유동주식비율,calc_유동주식수
ticker_name,,,,,,
삼성전자,005930,5969782550,4525870769,76,76,4525692151
삼성바이오로직스,207940,71174000,17623664,25,25,17622682
SK하이닉스,000660,728002365,537759511,74,74,537775347
LG화학,051910,70592343,47036786,67,67,47035678
삼성SDI,006400,68764530,50306427,73,73,50308130


In [9]:
4525870769/5969782550

0.75812992032683

# 일자별주가(종가)

In [36]:
df_list = []
for ticker in top_idx_list:
    ohlcv_df = stock.get_market_ohlcv(__START_DATE, __END_DATE, ticker)
    ohlcv_df = ohlcv_df.iloc[:, 3:4]
    df_list.append(ohlcv_df)
    
일자별주가 = pd.concat(df_list, axis=1)
일자별주가.columns = ticker_name_list  
일자별주가.head()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [11]:
일자별주가.to_csv("일자별주가.csv")

In [12]:
일자별주가.index.min(),일자별주가.index.max()

(Timestamp('2020-01-02 00:00:00'), Timestamp('2022-12-29 00:00:00'))

In [13]:
일자별주가.shape

(742, 5)

In [14]:
423485 * 17623664	

7463357349040

# 일자별 시가총액

In [15]:
일자별시가총액 = 일자별주가 * 종목기본정보['유동주식수']
일자별시가총액.head()    

,삼성전자,삼성바이오로직스,SK하이닉스,LG화학,삼성SDI
날짜,,,,,
2020-01-02,249828066448800,7463357349040,50925825691700,14769550804000,11671091064000
2020-01-03,251185827679500,7350142931504,50818273789500,14628440446000,11520171783000
2020-01-06,251185827679500,7175968260192,50710721887300,14440293302000,11570478210000
2020-01-07,252543588910200,7297888767744,50549394034000,14628440446000,11645937850500
2020-01-08,257069459679200,7115008006416,52377776371400,14205109372000,11520171783000


In [16]:
일자별시가총액["합계"] = 일자별시가총액.sum(axis=1)
일자별시가총액["지수"] = (일자별시가총액["합계"] / 일자별시가총액.iloc[0]["합계"] ) * 100
일자별시가총액.head()

,삼성전자,삼성바이오로직스,SK하이닉스,LG화학,삼성SDI,합계,지수
날짜,,,,,,,
2020-01-02,249828066448800,7463357349040,50925825691700,14769550804000,11671091064000,334657891357540,100
2020-01-03,251185827679500,7350142931504,50818273789500,14628440446000,11520171783000,335502856629504,100
2020-01-06,251185827679500,7175968260192,50710721887300,14440293302000,11570478210000,335083289338992,100
2020-01-07,252543588910200,7297888767744,50549394034000,14628440446000,11645937850500,336665250008444,101
2020-01-08,257069459679200,7115008006416,52377776371400,14205109372000,11520171783000,342287525212016,102


# 랜덤 설정좌수, 환매좌수
* 실습07_펀드_랜덤좌수.ipynb 참조

In [17]:
temp = pd.read_csv("./실습07_설정환매_랜덤.csv", parse_dates=['날짜'])
temp = temp.set_index("날짜")
temp.head()

,설정,환매
날짜,,
2020-01-02,1000000,0
2020-01-03,50000,0
2020-01-06,150000,100000
2020-01-07,100000,100000
2020-01-08,150000,100000


# 편입비중

In [18]:
pd.options.display.float_format = '{:.4f}'.format
rate_df = pd.DataFrame() 
for idx in 일자별시가총액.index:
    s = 일자별시가총액.loc[idx] / 일자별시가총액.loc[idx,'합계']
    rate_df[idx] = s

rate_df = rate_df.T
rate_df = pd.merge(rate_df, temp, left_index=True, right_index=True)
#rate_df = pd.concat([rate_df, temp], axis=1)
rate_df['지수'] = 일자별시가총액['지수']
rate_df.head()

,삼성전자,삼성바이오로직스,SK하이닉스,LG화학,삼성SDI,합계,지수,설정,환매
2020-01-02,0.7465,0.0223,0.1522,0.0441,0.0349,1.0000,100.0000,1000000,0
2020-01-03,0.7487,0.0219,0.1515,0.0436,0.0343,1.0000,100.2525,50000,0
2020-01-06,0.7496,0.0214,0.1513,0.0431,0.0345,1.0000,100.1271,150000,100000
2020-01-07,0.7501,0.0217,0.1501,0.0435,0.0346,1.0000,100.5998,100000,100000
2020-01-08,0.7510,0.0208,0.1530,0.0415,0.0337,1.0000,102.2798,150000,100000


In [28]:
rate_df['총좌수'] = (rate_df['설정'] - rate_df['환매']).cumsum()
# 지수변동률 * 기준가
rate_df['iNAV'] = (rate_df['지수'] / rate_df.iloc[0]['지수']) * 10000
rate_df['AUM']  = rate_df['총좌수'] * rate_df['iNAV'] 
rate_df.head()

,삼성전자,삼성바이오로직스,SK하이닉스,LG화학,삼성SDI,합계,지수,설정,환매,총좌수,iNAV,AUM
2020-01-02,0.7465,0.0223,0.1522,0.0441,0.0349,1.0000,100.0000,1000000,0,1000000,10000.0000,10000000000.0000
2020-01-03,0.7487,0.0219,0.1515,0.0436,0.0343,1.0000,100.2525,50000,0,1050000,10025.2486,10526511059.7561
2020-01-06,0.7496,0.0214,0.1513,0.0431,0.0345,1.0000,100.1271,150000,100000,1100000,10012.7114,11013982571.2072
2020-01-07,0.7501,0.0217,0.1501,0.0435,0.0346,1.0000,100.5998,100000,100000,1100000,10059.9824,11065980649.8821
2020-01-08,0.7510,0.0208,0.1530,0.0415,0.0337,1.0000,102.2798,150000,100000,1150000,10227.9831,11762180547.9338


In [29]:
rate_df.tail()

,삼성전자,삼성바이오로직스,SK하이닉스,LG화학,삼성SDI,합계,지수,설정,환매,총좌수,iNAV,AUM
2022-12-23,0.6938,0.0375,0.1104,0.0757,0.0826,1.0000,113.2436,50000,0,37450000,11324.3555,424097111921.0647
2022-12-26,0.6925,0.0383,0.1094,0.0768,0.0830,1.0000,113.0713,150000,100000,37500000,11307.1290,424017338655.6552
2022-12-27,0.6922,0.0384,0.1090,0.0778,0.0826,1.0000,113.5139,200000,100000,37600000,11351.3917,426812326621.5884
2022-12-28,0.6918,0.0395,0.1104,0.0765,0.0819,1.0000,110.6541,100000,50000,37650000,11065.4068,416612565417.2785
2022-12-29,0.6894,0.0399,0.1111,0.0777,0.0819,1.0000,108.4794,100000,50000,37700000,10847.9383,408967273060.8607


# 일자별 매매내역

In [30]:
펀드 = rate_df.copy()

In [31]:
펀드['AUM_diff'] = 펀드['AUM'].diff()
펀드['AUM_diff'].fillna(10000000000, inplace=True)
펀드.head()

,삼성전자,삼성바이오로직스,SK하이닉스,LG화학,삼성SDI,합계,지수,설정,환매,총좌수,iNAV,AUM,AUM_diff
2020-01-02,0.7465,0.0223,0.1522,0.0441,0.0349,1.0000,100.0000,1000000,0,1000000,10000.0000,10000000000.0000,10000000000.0000
2020-01-03,0.7487,0.0219,0.1515,0.0436,0.0343,1.0000,100.2525,50000,0,1050000,10025.2486,10526511059.7561,526511059.7561
2020-01-06,0.7496,0.0214,0.1513,0.0431,0.0345,1.0000,100.1271,150000,100000,1100000,10012.7114,11013982571.2072,487471511.4512
2020-01-07,0.7501,0.0217,0.1501,0.0435,0.0346,1.0000,100.5998,100000,100000,1100000,10059.9824,11065980649.8821,51998078.6749
2020-01-08,0.7510,0.0208,0.1530,0.0415,0.0337,1.0000,102.2798,150000,100000,1150000,10227.9831,11762180547.9338,696199898.0516


In [32]:
for ticker in ticker_name_list:
    펀드[f"매매_{ticker}"] = np.trunc(펀드['AUM_diff'] * 펀드[ticker] / 일자별주가[ticker])
    펀드[f"잔고_{ticker}"] = 펀드[f"매매_{ticker}"].cumsum()
펀드.head()

,삼성전자,삼성바이오로직스,SK하이닉스,LG화학,삼성SDI,합계,지수,설정,환매,총좌수,iNAV,AUM,AUM_diff,매매_삼성전자,잔고_삼성전자,매매_삼성바이오로직스,잔고_삼성바이오로직스,매매_SK하이닉스,잔고_SK하이닉스,매매_LG화학,잔고_LG화학,매매_삼성SDI,잔고_삼성SDI
2020-01-02,0.7465,0.0223,0.1522,0.0441,0.0349,1.0000,100.0000,1000000,0,1000000,10000.0000,10000000000.0000,10000000000.0000,135238.0000,135238.0000,526.0000,526.0000,16068.0000,16068.0000,1405.0000,1405.0000,1503.0000,1503.0000
2020-01-03,0.7487,0.0219,0.1515,0.0436,0.0343,1.0000,100.2525,50000,0,1050000,10025.2486,10526511059.7561,526511059.7561,142000.0000,277238.0000,552.0000,1078.0000,16872.0000,32940.0000,1475.0000,2880.0000,1578.0000,3081.0000
2020-01-06,0.7496,0.0214,0.1513,0.0431,0.0345,1.0000,100.1271,150000,100000,1100000,10012.7114,11013982571.2072,487471511.4512,148762.0000,426000.0000,579.0000,1657.0000,17675.0000,50615.0000,1546.0000,4426.0000,1653.0000,4734.0000
2020-01-07,0.7501,0.0217,0.1501,0.0435,0.0346,1.0000,100.5998,100000,100000,1100000,10059.9824,11065980649.8821,51998078.6749,148762.0000,574762.0000,579.0000,2236.0000,17675.0000,68290.0000,1546.0000,5972.0000,1653.0000,6387.0000
2020-01-08,0.7510,0.0208,0.1530,0.0415,0.0337,1.0000,102.2798,150000,100000,1150000,10227.9831,11762180547.9338,696199898.0516,155524.0000,730286.0000,605.0000,2841.0000,18479.0000,86769.0000,1616.0000,7588.0000,1728.0000,8115.0000


In [33]:
펀드.tail(1)

,삼성전자,삼성바이오로직스,SK하이닉스,LG화학,삼성SDI,합계,지수,설정,환매,총좌수,iNAV,AUM,AUM_diff,매매_삼성전자,잔고_삼성전자,매매_삼성바이오로직스,잔고_삼성바이오로직스,매매_SK하이닉스,잔고_SK하이닉스,매매_LG화학,잔고_LG화학,매매_삼성SDI,잔고_삼성SDI
2022-12-29,0.6894,0.0399,0.1111,0.0777,0.0819,1.0000,108.4794,100000,50000,37700000,10847.9383,408967273060.8607,-7645292356.4178,5098500.0000,1943853485.0000,19853.0000,7568974.0000,605798.0000,230966425.0000,52988.0000,20201849.0000,56671.0000,21606143.0000


In [ ]:
펀드.to_csv("검증.csv")